In [19]:
import datetime
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [ ]:
try:
    from catboost import Pool, CatBoostRegressor, cv
except:
    !pip install catboost 
    from catboost import Pool, CatBoostRegressor, cv

In [ ]:
try:
    from CustomPipeline import *
except:
    import sys
    sys.path.insert(0,'/content/sample_data')
    from CustomPipeline import *
    print("ok")

In [20]:
# test = pd.read_csv("./test.csv", delimiter=",", sep='.')
# train = pd.read_csv("./train.csv", delimiter=",", sep='.')

# test = pd.read_csv("./test_anomaly.csv", delimiter=",", sep='.')
# train = pd.read_csv("./train_anomaly.csv", delimiter=",", sep='.')

try:
    train = pd.read_csv("./train_anomaly.csv", delimiter=",", sep='.')
except:
    train = pd.read_csv("./sample_data/train_anomaly.csv", delimiter=",", sep='.')

In [21]:
pd.set_option('display.max_columns', None)
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,anomaly_col,cont1_clusters
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,Norm,cluster_low
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,Anomaly,cluster_low
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,Anomaly,cluster_low
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,Norm,cluster_high
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,Norm,cluster_low


In [ ]:
X = train.drop(["target", "id"], axis=1)
y = train["target"]

In [23]:
num_train = X.select_dtypes([int, float])
cat_train = X.select_dtypes(object)

num = list(num_train)
cat = list(cat_train)

idx = [X.columns.get_loc(i) for i in cat]

In [28]:
train_pool = Pool(X, y, cat_features=idx)

def objective(trial):

#     param = {
#         "iterations": 200, 
#         "loss_function": "RMSE",
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "depth": trial.suggest_int("depth", 1, 12),
#         "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
#         ),
#         "used_ram_limit": "3gb",
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    param = {
        'iterations' : trial.suggest_int('iterations', 100, 500),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                                    
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 0.5),
    }
    param["loss_function"] ="RMSE"
    
    rmse = cv(train_pool, param, fold_count=5, plot=False, logging_level="Silent")

    return rmse['test-RMSE-mean'].iloc[-1]

In [ ]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20, n_jobs=-1)

[I 2021-08-20 19:11:00,419] A new study created in memory with name: no-name-11035aae-3115-43a8-aa50-6b9fa5496964


In [ ]:
best_params = study.best_trial.params
best_params

# save configs

In [ ]:
import yaml
import os


directory = './Configs'
if not os.path.exists(directory):
    os.makedirs(directory)
    
    
with open('./Configs/params_model_catboost.yml', 'w') as outfile:
    yaml.dump(best_params, outfile, default_flow_style=False)